# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: Identifying students who might need early intervention before they fail to graduate requires classifying whether the student will fail to graduate or graduate. This classification will have to depend on either features that discrete(categorical or binary) or continuous such as grade point average, attendance, or acessibility to resources(such as internet). If we supposed this was a regression problem and we were to just analyze one of the continuous variables such attendance(as a percentage) and put it on the x-axis of our graph, what goes on our y-axis? Necesarily, the y-axis would have to be our target variable which is whether or not the student graduated. The only problem is that the y-axis would be a discrete variable and the x-axis would be a continous variable. For the regression model to work, both the variables on the y and x axis would have to be continous and therefore this must be a classification problem.**

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [2]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [3]:
# TODO: Calculate number of students
n_students = student_data.shape[0] 

# TODO: Calculate number of features
n_features = len(list(student_data.columns[:-1]))

# TODO: Calculate passing students
n_passed = student_data[student_data["passed"]=="yes"].shape[0]
#print student_data[student_data["passed"]=="yes"]


# TODO: Calculate failing students
n_failed = student_data[student_data["passed"]=="no"].shape[0]

# TODO: Calculate graduation rate
grad_rate = (float(n_passed)/float(n_students))*100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [4]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [5]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [6]:
# TODO: Import any additional functionality you may need here
from sklearn import cross_validation

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train
# TODO: Shuffle and split the dataset into the number of training and testing points above

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_all,y_all,test_size=0.2405,random_state=0)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: The three supervised learning models to be used are K-Nearest Neighbors or KNN, Support Vector Machines of SVM, and Random Forests (from Ensemble Learning). 
The KNN algorithm is typically used within Recommender Systems (refer to https://www.quora.com/What-are-industry-applications-of-the-K-nearest-neighbor-algorithm) for targeted marketing. The KNN model is intuitive due to it's concept of classifying a sample point based on the characterisitcs of it's "k" nearest neighbors. In practice, tuning the model is quite easy as the main concern is choosing what "k" to use. The KNN algorithm runs in O(ND+NK) (this assumes you compute all the distances and store them within memory) time where D is the dimensionality of the feature space, N is the number of training data points, and K is the amount of nearest neighbors to consider. Although the training time is essentially zero (besides the amount of time it takes for the computer to store all the training points in memory), the prediction step is computationally expensive. It's also important to note that while the training step is not computationally expensive, it quickly becomes memory intensive as the N goes up. Although memory and computationally expensive, the main utility of the KNN model in regards to the problem at hand lies within it's ease of tuning (since there is only one main parameter to consider usually) and it's lack of assumptions(other than the fact that the feature space has a measure of similarity, e.g Euclidean distance). The con of being resource intensive is alleviated with the fact that faster computer processing power and larger memory reserves becomes cheaper and cheaper as time goes on. Assuming that the model gets more training data once a year with a new graduating class the size of approximately 395 students, the model will have to be retrained at least once a year (where retraining only requires that the new inputs can fit in memory). Adding more data in the order of hundreds can easily be handled by today's methods of data storage. Prediction on the other hand will most likely be run multiple times throughout the year to track the students' progress, which can be computationally expensive once the model has training data in order of thousands. With modern computers the computation may take a few minutes or a few hours once the quantities of the training and testing data sets become sufficiently large, but predictions are not required in real-time. Corrective action can still be taken for students even if the computer takes a day to get the results given that these efforts to track progress were implemented well before a quarter or semester ends. 
One real world application of Support Vector Machines is spam detection as shown in the following link: https://www.r-bloggers.com/build-a-spam-filter-with-r/. Some strengths include being able to handle non-linearly separable data well (with the use of the kernel trick), effective in cases where dimensionality of data is greater than the number of samples, memory efficient due to using only support vectors to determine the decision function and customizable (requires domain knowledge). The main difficulty with the SVM model is it's double-edged characteristic, customizability. SVMs are sensitive to the type of kernel used and also the parameters within each kernel themselves which makes the usability learning curve steep. In practice, this weakness is addressed with methods such as Grid Search which automates optimizing the hyperparameters such as kernel type and kernel parameters. Another disadvantage that comes up is both the training and testing phases can be quite slow, especially if you're also trying to tune the model using Grid Search. However, as mentioned before, real-time results are not required and the retraining of the model need only occur about once a year. The student data has a high dimensional feature space which the SVM model is adept at dealing with and it's memory efficient in that it learns a decision function (linear or non-linear) unlike the KNN algorithm. 
Like the KNN and SVM algorithm, the Random Forests model also deals with non-linear dependencies well. One real life application of the algorithm is analysis of mass spectrometry-based proteomics data as shown in the following link: http://www.cs.cmu.edu/~qyj/papersA08/11-rfbook.pdf. One main advantage the RF model posseses is speed, it's unlikeliness to overfit the data, and it's ease of tuning similarly to the KNN model. The RF model's overall process is quite intuitive but the actual inner workings are treated more like a black box similar to SVM. It works well if you need a classification of test points but it often doesn't provide much insight on what features and other aspects of the data are important. From experience, the results for Random Forest can vary quite a bit between each run of the algorithm due the random sampling that occurs with the data set during each iteration, unless you set a random state. This is especially true when the feature space has many dimensions and the amount of training points is not sufficiently large. While Random Forest may not be sufficient for the student intervention model with the given number of data points and the current dimensionality of the feature space, it is still a running candidate when the number of dataset size increases to the order of thousands and SVM and KNN become progressively more expensive and slow to run. **

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [7]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [16]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
# from sklearn import model_B
# from sklearn import model_C

from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import numpy as np 


# TODO: Initialize the three models
clf_A = KNeighborsClassifier()
clf_B = svm.SVC()
clf_C = RandomForestClassifier(random_state=39)

# TODO: Set up the training set sizes
X_train_100, X_test_100, y_train_100, y_test_100 = cross_validation.train_test_split(X_all,y_all,test_size=0.7468,random_state=0)
X_train_200, X_test_200, y_train_200, y_test_200 = cross_validation.train_test_split(X_all,y_all,test_size=0.49367,random_state=0)
X_train_300, X_test_300, y_train_300, y_test_300 = cross_validation.train_test_split(X_all,y_all,test_size=0.2405,random_state=0)

# TODO: Execute the 'train_predict' function for each classifier and each training set size
train_predict(clf_A, X_train_100, y_train_100, X_test_100, y_test_100)
train_predict(clf_A, X_train_200, y_train_200, X_test_200, y_test_200)
train_predict(clf_A, X_train_300, y_train_300, X_test_300, y_test_300)

train_predict(clf_B, X_train_100, y_train_100, X_test_100, y_test_100)
train_predict(clf_B, X_train_200, y_train_200, X_test_200, y_test_200)
train_predict(clf_B, X_train_300, y_train_300, X_test_300, y_test_300)

train_predict(clf_C, X_train_100, y_train_100, X_test_100, y_test_100)
train_predict(clf_C, X_train_200, y_train_200, X_test_200, y_test_200)
train_predict(clf_C, X_train_300, y_train_300, X_test_300, y_test_300)
# train_predict(clf, X_train, y_train, X_test, y_test)

Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0010 seconds
Made predictions in 0.0037 seconds.
F1 score for training set: 0.8571.
Made predictions in 0.0051 seconds.
F1 score for test set: 0.7780.
Training a KNeighborsClassifier using a training set size of 200. . .
Trained model in 0.0016 seconds
Made predictions in 0.0056 seconds.
F1 score for training set: 0.8635.
Made predictions in 0.0074 seconds.
F1 score for test set: 0.7639.
Training a KNeighborsClassifier using a training set size of 300. . .
Trained model in 0.0010 seconds
Made predictions in 0.0062 seconds.
F1 score for training set: 0.8722.
Made predictions in 0.0021 seconds.
F1 score for test set: 0.7482.
Training a SVC using a training set size of 100. . .
Trained model in 0.0013 seconds
Made predictions in 0.0007 seconds.
F1 score for training set: 0.8690.
Made predictions in 0.0016 seconds.
F1 score for test set: 0.7824.
Training a SVC using a training set size of 200. . .
Trai

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - K-Nearest Neighbors Classifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0034 s                | 0.0090 s               |0.8571 |0.7780 |
| 200               |0.0036 s                 |0.0033 s                |0.8635|0.7639  |
| 300               |0.0016 s                 |0.0031 s                |0.8722|0.7482  |

** Classifer 2 - Support Vector Machine **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |0.0019 s                 |0.0016 s                |0.8690|0.7824     |
| 200               |0.0035 s  | 0.0021 s     |0.8765                  |0.7781            |
| 300               | 0.0123 s | 0.0016 s    | 0.8693                  |   0.7586         |

** Classifer 3 - Random Forest Classifier **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100       |   0.0255 s   | 0.0011 s  |1.000   |0.7050   |
| 200           |    0.0292 s|0.0011 s  |0.9860        |0.7312    |                
| 300     | 0.0272 s  | 0.0011 s  | 0.9904  |0.7273           |                 

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: The SVM model scored the highest average F1 prediction score of 0.77303 compared to 0.7631 and 0.7212 of the KNN and RF model respectively. This average was calculated using the simple arithmetic mean formula across the three training sets of 100, 200, and 300 data points. It's important to note that due to the random sampling of the data trained on that occurs each iteration of the RF model, the F1 scores tend to vary in the range of 0.69 - 0.80 each time you retrain without setting random state. This variation indicates that the number of training points is not sufficiently large enough to compensate for the dimensionality of the data. Although one can obtain consistently high F1 prediction scores by setting a particular random state, it's not certain the model would generalize well to new data. The KNN model's average F1 prediction score was comparable to that of the SVM's but nonetheless lower. It is also known from experience and research that KNN does not generalize as accurately as SVM in higher dimensional feature spaces such as this one. 
The conclusion is that in the beginning stages of this new student intervention program, the SVM model shall be used but as time goes on, the RF model needs to be considered. Inevitably, the data set will grow larger as more students graduate each year and the concern is that SVM will become computationally expensive and too time consuming to run. As the data set size scales up, the RF model's variation in accuracy or F1 score in this case should begin to stabilize and it's computation speed begins to outpace that of the SVM and KNN models (empirically seen in commercial use and research settings) even though it took the longest time to train in this recent experiment.**

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: The SVM firsts takes the student training data and esentially maps the data on a graph where the number of the characteristics that descibes the students also is the number of axes. The model then attempts to define a boundary that separates the students that have failed to graduate from those who have. In real life there are many instances where defined border doesn't perfectly separate the data into the distinct categories due to noise and outliers within the collected data set.It's important to understand that imperfect separation of the data isn't necessarily a bad thing if the data is faulty to begin with. The SVM model will make a tradeoff between minimizing misclassifcations and sufficient model generalization. 
In order to understand how this tradeoff is implemented, the following allegory will be told.Imagine that there are two countries, Country A and Country B, are the only land masses on this Earth and that we have have a map that depicts. However, the map does not depict the border between the two countries. It's unbelievable that a map maker could make such a mistake as you know a border certainly exists! From our demographic studies, we know that both of these countries contain areas dense with real estate. Assume that each defined cluster only contains peoples from either Country A or Country B, never both. One makes the assumption that when a cluster from Country A is really close to a cluster from Country B (or vice versa), with no other clusters in between these two, these clusters must lie near the border. This also makes the assumption that no clusters from either country will lie directly on the border. It's also quite easy to distinguish whether one cluster belongs to Country A or Country B due to the distinct physical and cultural characteristics of each one. 
In regards to implementation, the midpoint between a cluster from Country A to all other close by clusters of country B will be recorded. As discussed earlier, calculations will only be made between clusters from different countries that are close to each other. This means that pairs of  clusters clearly from the same country in close proximity will not have their midpoints calculated since it's unlikely for a border to be between clusters with peoples from the same country. The complications comes in when you realize that clusters of from Country A have moved and settled in Country B. Since it isn't known where the true borders are, it's uncertain whether any cluster is actually part of Country A or Country B. Intuitively, if a cluster of people from Country A surrounded on many sides by clusters of Country B then it would be safe to assume that it's a cluster from Country A that settled in Country B. The same reasoning can be applied to a cluster from Country B that is surrounded on many sides by clusters from Country A. These would be the outliers in our model that may prevent us from creating a border that perfectly separates clusters with peoples from Country A from Country B. In the case that a cluster from Country A migrated to Country B but was just a few feet from the border, then it would be unclear whether the cluster from Country A still resided in Country A or was actually in Country B territory. 
Our goal of achieving a sufficient estimation of the border can be reached by minimizing the amount of misclassified clusters, which includes the outliers, and using the midpoint between pairs of clusters(where pairs must consist of clusters with clearly different types of peoples). It should be noted that using the midpoint between a pair of clusters allows unbiased allocation of space. 
Going back to our student training data set, replace Country A with the students who are labeled as graduated and Country B with the students who are labeled as not graduated and you should have a good sense of how Support Vector Machine functions. The added complexity here is that the allegory only considered a two dimensional map. The "map" that the students are located on will be multidimensional. When describing the location of cluster of people on a traditional map, you would just need latitude and longitude (two dimensions). What makes our problem multidimensional is the fact that in our data set, a student is described by many features such as their sex, the school they go to, whether they are in a relationship, etc. Once this border between students who did and didn't graduate is determined, the classification for any new student would be based on what side of the that border they lie on.**

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [ ]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
#from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer,f1_score
from sklearn import svm
import numpy as np 
from sklearn.grid_search import GridSearchCV

# TODO: Create the parameters list you wish to tune
parameters = {'kernel':('linear','rbf'), 'C':[0.00001,0.0001,0.001,0.01,1,10,100,1000], 'gamma':[0.0001,0.001,0.01,0.1,1,10,100,1000]}

# TODO: Initialize the classifier
clf = svm.SVC()
#print clf.get_params()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label = 'yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring = f1_scorer, cv=5, n_jobs = 100)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_
print clf.get_params

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer:  The tuned model's F1 score for training and testing are 0.8200 and 0.7867, respectively, compared to 0.8693 and 0.7586 of the untuned model. The tuned model’s training score decreased by 5.6% while it’s training score increased by 3.7%. The untuned model uses an RBF kernel with a C parameter of 100  while the tuned model employs the Linear kernel with a C parameter of 1 (the default value for all the kernels in the sklearn library). As the C parameter increases,the SVM model penalizes misclassifications correspondingly more. As a result, the variance of the model also increases as the bias decreases. One must take caution in choosing C as to not overfit the training data. Luckily, using grid search and cross validation procedures alleviates this risk. The opposite reasoning is applied to the scenario that occurs when the C parameter decreases. Seeing that the tuned model used the RBF kernel, the gamma parameter (was not present in the Linear Kernel) must also be considered. The gamma parameter controls the variance of the RBF kernel function. Scaling gamma down increases the model's variance and thus reduces the bias. Scaling gamma up does the opposite. Going back to the tuned model, the RBF kernel used a gamma parameter of 0.0001 and the mentioned C parameter of 100. After tuning, the SVM model increased its variance while reducing bias. The untuned model has a higher bias and is less "sensitive" to the variance of the student data set. The hyperplane defined by the tuned model was less "smooth" than that of the untuned model. There was more contortion in the tuned model's border between those who passed and those who did not. The appropriate bias-variance tradeoff achieved by grid search and cross-validation led to the training score increase of 3.7%.**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.